In [2]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [3]:
import sys
sys.path.insert(0, MAIN_PATH)

In [8]:
import pandas as pd

In [5]:
%load_ext autoreload
%autoreload 2


%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Métricas

El objetivo de este notebook es reunir las métricas que se obtienen a partir de las distintas técnicas de resuménes extractivos que se han 
desarrollado.

En esta primera parte, el objetivo ha sido realizar un resumen de los partidos, utilizando directamente los eventos. Por tanto, para evaluar
la bondad de estos métodos, se comparará directamente con la noticia asociada a cada partido, (la cual podemos considerar resumen del partido)
utilizando las siguientes métricas:

- ROUGE: mide el recall. Cuántas secuencias (unigramas, bigramas, lcs...) del resumen generado se encuentran en el resumen real, del total de las secuencias
del resumen real.
- BLEU: mide la precision. Cuántas secuencias del resumen generado se encuentran en el resumen real, del total de las secuencias del resumen generado.

Es importante considerar que no se está haciendo un resumen extractivo al uso (tradicionalmente se realizaría un resumen a partir de la noticia, 
y de ahí se extraerían las métricas). El estilo de escritura y los términos usados en eventos y noticias son bastante diferentes,
por lo que las métricas tradicionales obtenidas no serán demasiado altas. Nos interesa la comparación entre técnicas propuestas.

In [23]:
summaries_path = '{}summaries/key_events_summaries_2.csv'.format(CSV_DATA_PATH)
articles_path = '{}articles_events.csv'.format(CSV_DATA_PATH)

In [12]:
pd_summaries = pd.read_csv(summaries_path)
pd_matches = pd.read_csv(articles_path)

In [39]:
join_cols = ['json_file', 'url']
match_cols = join_cols + ['article', 'events']
sum_cols = join_cols + ['summary_events']

In [40]:
pd_matches_sum = pd_matches[match_cols].merge(pd_summaries[sum_cols], on=join_cols, how='inner')

In [71]:
pd_matches_sum

,json_file,url,article,events,summary_events
0,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49871134,Mario Balotelli scored his first Serie A goal ...,['Attempt blocked. Mario Balotelli (Brescia) r...,"['Goal! Napoli 1, Brescia 0. Dries Mertens (N..."
1,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49873763,Fiorentina condemned AC Milan to a third conse...,"['Foul by Nikola Milenkovic (Fiorentina).', 'R...","['Goal! Milan 0, Fiorentina 1. Erick Pulgar (..."
2,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49865134,Cristiano Ronaldo returned from injury with a ...,"['Corner, SPAL. Conceded by Matthijs de Ligt....","['Goal! Juventus 1, SPAL 0. Miralem Pjanic (J..."
3,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49866723,Alexis Sanchez scored and was sent off on his ...,"['Offside, Inter Milan. Milan Skriniar tries a...","['Goal! Sampdoria 0, Inter Milan 1. Stefano S..."
4,italian_serie_a_2019_2020.json,https://www.bbc.com/sport/football/49832309,On-loan Manchester United defender Chris Small...,['Attempt missed. Alejandro Gómez (Atalanta) r...,"['Goal! Roma 0, Atalanta 1. Duván Zapata (Ata..."
...,...,...,...,...,...
1674,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,The Week 19 matchup at Rio Tinto Stadium betwe...,['Foul by Raymon Gaddis (Philadelphia Union).'...,['VAR: Red Card. Referee decision on field sta...
1675,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,New York City FC extended its string of result...,"['Corner, New York City FC. Conceded by Zacha...",['VAR: Goal. Referee decision on field cancell...
1676,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,"Try as they might, Orlando City just can't sol...",['Attempt missed. Dom Dwyer (Orlando City SC) ...,['VAR: Goal. Referee decision on field cancell...
1677,mls_2019_2020.json,https://matchcenter.mlssoccer.com/matchcenter/...,"Diego Rossi scored his first MLS hat trick, Ca...",['Lucas Rodríguez (D.C. United) wins a free ki...,"['VAR: Red Card.', 'VAR: Red Card. Referee dec..."


In [49]:
prueba = pd_matches_sum.iloc[0]

In [51]:
prueba['article']

"Mario Balotelli scored his first Serie A goal for 1,468 days but Napoli hung on to claim a narrow victory over Brescia at Stadio San Paolo.\nDries Mertens opened the scoring with his fourth league goal of the season before Kostas Manolas headed in the second on the stroke of half-time. \nBrescia rallied after the break but Sandro Tonali's stunning goal was ruled out by VAR for a foul in the build-up.\nBalotelli headed home to set up a tense finale, but Napoli held on.\nThe 29-year-old last found the net in Italy's top flight on 22 September 2015 in AC Milan's 3-2 win at Udinese, before spells at Nice and Marseille in France.\nHe returned to Italy to join hometown club Brescia in August on a three-year deal.\nThe result sees Napoli strengthen their grip on fourth place in Serie A, while Brescia remain 14th.\nCarlo Ancelotti's side resume their Champions League campaign at Genk on Wednesday, 2 October (17:55 BST), before Brescia face Sassuolo in Serie A on 4 October (19:45).\nElsewhere 

In [55]:
prueba['events']

"['Attempt blocked. Mario Balotelli (Brescia) right footed shot from outside the box is blocked.', 'Corner,  Napoli. Conceded by Stefano Sabelli.', 'Attempt missed. Mario Balotelli (Brescia) left footed shot from the left side of the box misses to the right. Assisted by Nikolas Spalek.', 'Attempt saved. Mario Balotelli (Brescia) right footed shot from outside the box is saved in the bottom right corner. Assisted by Sandro Tonali.', 'Goal!  Napoli 1, Brescia 0. Dries Mertens (Napoli) right footed shot from the centre of the box to the bottom right corner. Assisted by José Callejón.', 'Corner,  Napoli. Conceded by Andrea Cistana.', 'Attempt saved. Fernando Llorente (Napoli) left footed shot from the centre of the box is saved in the bottom left corner. Assisted by Allan.', 'Corner,  Napoli. Conceded by Jesse Joronen.', 'Corner,  Napoli. Conceded by Andrea Cistana.', 'Attempt missed. Giovanni Di Lorenzo (Napoli) header from the centre of the box misses to the left. Assisted by José Callej

In [53]:
prueba['summary_events']

"['Goal!  Napoli 1, Brescia 0. Dries Mertens (Napoli) right footed shot from the centre of the box to the bottom right corner. Assisted by José Callejón.', 'GOAL CANCELLED BY VAR: Kostas Manolas (Napoli) scores but the goal is ruled out after a VAR review.', 'VAR Decision: No Goal  Napoli 1-0 Brescia.', 'Goal!  Napoli 2, Brescia 0. Kostas Manolas (Napoli) header from the centre of the box to the bottom right corner. Assisted by Nikola Maksimovic with a headed pass following a corner.', 'GOAL CANCELLED BY VAR: Sandro Tonali (Brescia) scores but the goal is ruled out after a VAR review.', 'VAR Decision: No Goal  Napoli 2-0 Brescia.', 'Goal!  Napoli 2, Brescia 1. Mario Balotelli (Brescia) header from the centre of the box to the top left corner. Assisted by Sandro Tonali with a cross following a corner.']"

In [ ]:
candidate = prueba['summary_events']